In [ ]:
# This needs to be modulized but works up until entering cc info

# TODO
* Works now up to clicking submit for credit card. 
* right now only one is getting booked which is the latest one. 
* need to modulize

## Initiate web site

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [4]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get("http://rivertownecountryclub.com/-online-tee-times")
# switch to iframe where all the info is displayed
find_i=browser.find_element(by=By.XPATH, value='//iframe')
WebDriverWait(browser,10).until(EC.frame_to_be_available_and_switch_to_it((find_i)))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//iframe"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
#0 0x55d696e7fcd3 <unknown>
#1 0x55d696c87968 <unknown>
#2 0x55d696cbefd7 <unknown>
#3 0x55d696cbf1a1 <unknown>
#4 0x55d696cf2154 <unknown>
#5 0x55d696cdcbdd <unknown>
#6 0x55d696cefea8 <unknown>
#7 0x55d696cdcaa3 <unknown>
#8 0x55d696cb23fa <unknown>
#9 0x55d696cb3555 <unknown>
#10 0x55d696ec72bd <unknown>
#11 0x55d696ecb418 <unknown>
#12 0x55d696eb136e <unknown>
#13 0x55d696ecc078 <unknown>
#14 0x55d696ea5bb0 <unknown>
#15 0x55d696ee8d58 <unknown>
#16 0x55d696ee8ed8 <unknown>
#17 0x55d696f02cfd <unknown>
#18 0x7fe00dee9609 <unknown>


## Add User Input

In [72]:
Desired_date='09-11-2022'
Desired_time='12:40 PM'
Desired_players=4
Number_of_Ttimes=2

In [83]:
# Create example User Data
from datetime import datetime
import time
f_name = 'Mike'
l_name = 'Bee'
email = 'mike@bee.com'
phone = '843-255-2656'
cc_num = '4266902068554848'
# Don't pad exp mon
Exp_mon = '4'
Exp_yr = '2024'
## Credit card options 
Visa='Visa'
Master='Master Card'
Discover='Discover'
Amex='American Express'

# Get All times for 7 day period and match criteria

In [74]:
# Loops through each day, grabs the table, adds the book url and tab location to df
# saves it to dictionray of key=date, value=df
import time
rivtown ={}
for d in range(7):
    day='daytab'+str(d)
    tab=browser.find_element(by=By.ID,value=day)
    tab.click()
    time.sleep(15)
    try:
        date=get_river_date(browser)
        df=get_rvdataframe(browser,date)
        df['Tab'] = day
        rivtown[date] = df
    except:
        print("No info for "+ day)
#     WebDriverWait(tab,10).click


# Find criteria matches

In [75]:
# filter based on input 
avail=rv_availbility(rivtown,Desired_date,Desired_time,Desired_players,Number_of_Ttimes)

# Book criteria matched

In [76]:
for i,a in avail.iterrows():
    book_tab=browser.find_element(by=By.ID,value=a['Tab'])
    # This has to be set after the tab click or it gets stale. 
#     booklink=browser.find_element(by=By.XPATH, value=a['BookLink'])

daytab6
daytab6


In [77]:
book_tab.click()
time.sleep(25)
booklink=browser.find_element(by=By.XPATH, value=a['BookLink'])
booklink.click()

In [78]:
# //*[@id="tee_time_table_container"]/table/tbody/tr[12]/td[5]/div/a/img

In [79]:
# booklink.click()

## Check out

In [80]:
ck_val='//*[@id="bookinginfo"]/div[2]/form/input[8]'
ck_out=browser.find_element(by=By.XPATH, value=ck_val)
# This will open a new window
ck_out.click()

In [81]:
# Find form window
original_window = browser.current_window_handle
for window_handle in browser.window_handles:
    if window_handle != original_window:
        browser.switch_to.window(window_handle)

In [82]:
fname = browser.find_element(by=By.XPATH, value='/html/body/center/div[1]/form/div[2]/table/tbody/tr[1]/td[2]/input')
lname = browser.find_element(by=By.XPATH, value='//*[@id="lname"]')
tel = browser.find_element(by=By.XPATH, value='//*[@id="phone"]')
mail = browser.find_element(by=By.XPATH, value='//*[@id="email"]')
card_num = browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/p[3]/input')

In [84]:
# Send Values to Form 
fname.send_keys(f_name)
lname.send_keys(l_name)
tel.send_keys(phone)
mail.send_keys(email)
card_num.send_keys(cc_num)

In [85]:
# Select Credit card type
cc_sel=browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/p[2]/select')
cc_sel.send_keys(Amex)

In [86]:
# Expiration 
cc_month=browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/p[4]/select[1]')
cc_year=browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/p[4]/select[2]')
cc_month.send_keys(Exp_mon)
cc_year.send_keys(Exp_yr)

In [87]:
# Cancellation policy 
accpt=browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/input')
accpt.click()

In [290]:
# submit 
submit=browser.find_element(by=By.XPATH, value='//*[@id="main"]/form/div[3]/p[6]/input')
submit.click()

In [10]:
def rv_availbility(crse_dict,Desired_date,Desired_time,Desired_players,Number_of_Ttimes):
    from datetime import datetime
    import time
    # convert to time objects
    date = datetime.strptime(Desired_date, '%m-%d-%Y').date().strftime("%A"+', '+"%B"+' '+"%d")
    time = datetime.strptime(Desired_time, '%I:%M %p').time()
    # grab df from dictionary
    df=crse_dict[date]
    # convert tee_time to datetime time function and save as filter col
    df['filter']= pd.to_datetime(df['Tee_Time']).dt.time
    # grab just the number from players col
    df['filter2']=df['Players'].str.split(' ',expand=False).str[0]
    # Grab all times greater than input time
    
    avail=df[(df['filter'] > time) & (df['filter2'] == str(Desired_players)) ]
    # grab the number of available times 
    avail_final=avail[:Number_of_Ttimes]
    return avail_final


In [54]:
# Add long wait
def get_river_time(browser):
    dt_val='//*[@id="tee_time_table_container"]/table/tbody/tr[1]/td[1]'
    date=browser.find_element(by=By.XPATH,value=dt_val).text
    date=date.split('\n')[1]
    return time
    
def get_river_date(browser):
    # Get Date
#     tbl_val='//*[@id="tee_time_table_container"]/table'
#     date=browser.find_element(by=By.XPATH,value=tbl_val)
#     dt_val='//*[@id="tee_time_table_container"]/table/tbody/tr[1]/td[1]/span'
    dt_val='//*[@id="tee_time_table_container"]/table/tbody/tr[1]/td[1]'
    date=browser.find_element(by=By.XPATH,value=dt_val).text
    date=date.split('\n')[0]
    return date

def get_rvdataframe(browser,date):
    from selenium.webdriver.common.by import By
    import pandas as pd
    
    def add_rvbook_url(row):
        bk_id=row.name+1
        rvb_url='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(bk_id)+']/td[5]/div/a/img'
        return rvb_url
    # Get Table
    tbl_val='//*[@id="tee_time_table_container"]/table'
    tbl=browser.find_element(by=By.XPATH,value=tbl_val)
    tee_html=pd.read_html(tbl.get_attribute('outerHTML'))
    tee_tble=tee_html[0]

    # Drop blank
    tee_tble=tee_tble.dropna(how='all').copy()
    # Split date and time into seperate columns
    tee_tble[['delete','Tee_Time']]=tee_tble['Tee Time'].str.split(date,expand=True)
    tee_tble['Tee_Date']= date
    tee_tble['18 Holes']=tee_tble['18 Holes'][1].split('18 Holes',1)[0]
    tee_tble['Local Rates']=tee_tble['Local Rates'][1].split('Local Rates',1)[0]
    # Select new columns and reset index 
    tee_tble=tee_tble[['Tee_Date','Tee_Time','Course','Players','18 Holes','Local Rates']]
    tee_tble=tee_tble.reset_index(drop=True)
    
    tee_tble['BookLink'] = tee_tble.apply(lambda row: add_rvbook_url(row), axis=1)
    return tee_tble

def add_rvbook_url(row):
    bk_id=row.name+1
#     b_url='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(bk_id)+']/td[4]/div/div[2]/a/img'
    rvb_url='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(bk_id)+']/td[5]/div/a/img'
    return rvb_url



In [12]:
def River_towne():
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    browser.get("http://rivertownecountryclub.com/-online-tee-times")
    # switch to iframe where all the info is displayed
    find_i=browser.find_element(by=By.XPATH, value='//iframe')
    WebDriverWait(browser,10).until(EC.frame_to_be_available_and_switch_to_it((find_i)))

def golf_day(rownum):
    # Tee Time 
    time_val = '//*[@id="tee_time_table_container"]/table/tbody/tr['+str(rownum)+']/td[1]/a'
    time=browser.find_element(by=By.XPATH, value=time_val)
    tee_time=str(time.get_attribute('outerHTML').split('>')[1].split('<')[0])
    # Date 
    date=str(time.get_attribute('onclick').split(',')[0].split('(')[1].strip("'"))
    hour=str(time.get_attribute('onclick').split(',')[2])
    minute=str(time.get_attribute('onclick').split(',')[3])
    # Course 
    crs_val='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(rownum)+']/td[2]/span'
    course=browser.find_element(by=By.XPATH,value=crs_val).text
    # Players
    play_val='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(rownum)+']/td[3]/span'
    players=browser.find_element(by=By.XPATH,value=play_val).text
    # Price 
    price_val='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(rownum)+']/td[4]/div/div[1]/span'
    price=browser.find_element(by=By.XPATH,value=price_val).text
    
    day_row = [date,tee_time,course,players,price]
    return day_row

In [20]:
def find_all_iframes(driver):
    iframes = driver.find_element(by=By.XPATH, value="//iframe")
    for index, iframe in enumerate(iframes):
        # Your sweet business logic applied to iframe goes here.
        driver.switch_to.frame(index)
        find_all_iframes(driver)
        driver.switch_to.parent_frame()

# Refactoring

## Fix time 

In [47]:
day='daytab2'
tab=browser.find_element(by=By.ID,value=day)
tab.click()

In [49]:
date=get_river_date(browser)
date
# df=get_rvdataframe(browser,date)
# df['Tab'] = day
# rivtown[date] = df

'Wednesday, September 7'

In [53]:
# Get Table
tbl_val='//*[@id="tee_time_table_container"]/table'
tbl=browser.find_element(by=By.XPATH,value=tbl_val)
tee_html=pd.read_html(tbl.get_attribute('outerHTML'))
tee_tble=tee_html[0]

# Split date and time into seperate columns
tee_tble[['delete','Tee_Time']]=tee_tble['Tee Time'].str.split(date,expand=True)
tee_tble['Tee_Date']= date
tee_tble['18 Holes']=tee_tble['18 Holes'][1].split('18 Holes',1)[0]
tee_tble['Local Rates']=tee_tble['Local Rates'][1].split('Local Rates',1)[0]
# Select new columns and reset index 
tee_tble=tee_tble[['Tee_Date','Tee_Time','Course','Players','18 Holes','Local Rates']]
tee_tble=tee_tble.reset_index(drop=True)
tee_tble


,Tee_Date,Tee_Time,Course,Players,18 Holes,Local Rates
0,"Wednesday, September 7",9:20 AM,RiverTowne Country Club,4 players,$99,$84
1,"Wednesday, September 7",10:30 AM,RiverTowne Country Club,1 players,$99,$84
2,"Wednesday, September 7",10:40 AM,RiverTowne Country Club,4 players,$99,$84
3,"Wednesday, September 7",10:50 AM,RiverTowne Country Club,4 players,$99,$84
4,"Wednesday, September 7",11:00 AM,RiverTowne Country Club,4 players,$99,$84
5,"Wednesday, September 7",11:10 AM,RiverTowne Country Club,4 players,$99,$84
6,"Wednesday, September 7",11:20 AM,RiverTowne Country Club,4 players,$99,$84
7,"Wednesday, September 7",11:30 AM,RiverTowne Country Club,4 players,$99,$84
8,"Wednesday, September 7",11:40 AM,RiverTowne Country Club,4 players,$99,$84
9,"Wednesday, September 7",11:50 AM,RiverTowne Country Club,4 players,$99,$84


In [47]:
def get_dataframe(browser,date):
    from selenium.webdriver.common.by import By
    import pandas as pd
    
    def add_book_url(row):
        bk_id=row.name+1
        b_url='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(bk_id)+']/td[4]/div/div[2]/a/img'
        return b_url
    # Get Table
    tbl_val='//*[@id="tee_time_table_container"]/table'
    tbl=browser.find_element(by=By.XPATH,value=tbl_val)
    tee_html=pd.read_html(tbl.get_attribute('outerHTML'))
    tee_tble=tee_html[0]

    # Drop blank
    tee_tble=tee_tble.dropna(how='all').copy()
    # Split date and time into seperate columns
    # unlike dunes data can be split on the new line character
#     tee_tble[['delete','Tee_Time']]=tee_tble['Tee Time'].str.split(date,expand=True)
    tee_tble['Tee_Date']= date.split('\n')[0]
    tee_tble['Tee_Time']= date.split('\n')[1]
    tee_tble['18 Holes']=tee_tble['18 Holes'][1].split('18 Holes',1)[0]
    tee_tble['Local Rates']=tee_tble['Local Rates'][1].split('Local Rates',1)[0]
    
    
    # Select new columns and reset index 
    tee_tble=tee_tble[['Tee_Date','Tee_Time','Course','Players','18 Holes','Local Rates']]
    tee_tble=tee_tble.reset_index(drop=True)
    
    tee_tble['BookLink'] = tee_tble.apply(lambda row: add_book_url(row), axis=1)
    return tee_tble

In [29]:
    tbl_val='//*[@id="tee_time_table_container"]/table'
    tbl=browser.find_element(by=By.XPATH,value=tbl_val)
    tee_html=pd.read_html(tbl.get_attribute('outerHTML'))
    tee_tble=tee_html[0]    
        # Drop blank
    tee_tble=tee_tble.dropna(how='all').copy()
    


In [ ]:
# Split date and time into seperate columns
tee_tble[['delete','Tee_Time']]=tee_tble['Tee Time'].str.split(date,expand=True)
tee_tble['Tee_Date']= date 

In [34]:
date.split('\n')[1]

'10:30 AM'

In [42]:
tee_tble['18 Holes'][1].split('18 Holes',1)[0]

'$99'

In [44]:
tee_tble['Local Rates'][1].split('Local Rates',1)[0]

'$84'

In [50]:
#         bk_id=row.name+1
#         b_url='//*[@id="tee_time_table_container"]/table/tbody/tr['+str(bk_id)+']/td[4]/div/div[2]/a/img'
b_url='//*[@id="tee_time_table_container"]/table/tbody/tr[1]/td[5]/div/a/img'
#         return b_url

In [56]:
booklink

AttributeError: 'WebElement' object has no attribute 'txt'

In [57]:
booklink=browser.find_element(by=By.XPATH, value=b_url)
booklink.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tee_time_table_container"]/table/tbody/tr[1]/td[5]/div/a/img"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
#0 0x55a279e2bcd3 <unknown>
#1 0x55a279c33968 <unknown>
#2 0x55a279c6afd7 <unknown>
#3 0x55a279c6b1a1 <unknown>
#4 0x55a279c9e154 <unknown>
#5 0x55a279c88bdd <unknown>
#6 0x55a279c9bea8 <unknown>
#7 0x55a279c88aa3 <unknown>
#8 0x55a279c5e3fa <unknown>
#9 0x55a279c5f555 <unknown>
#10 0x55a279e732bd <unknown>
#11 0x55a279e77418 <unknown>
#12 0x55a279e5d36e <unknown>
#13 0x55a279e78078 <unknown>
#14 0x55a279e51bb0 <unknown>
#15 0x55a279e94d58 <unknown>
#16 0x55a279e94ed8 <unknown>
#17 0x55a279eaecfd <unknown>
#18 0x7f709b1f8609 <unknown>
